In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: '../input/riiid-test-answer-prediction/split10/train_0.pickle'

In [ ]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [ ]:
w_df = df[["user_id", "task_container_id", "timestamp", "prior_question_elapsed_time"]]
w_df = w_df.drop_duplicates(["user_id", "task_container_id"])

In [ ]:
w_df["elapsed_time"] = w_df.groupby(["user_id"])["prior_question_elapsed_time"].shift(-1)

In [ ]:
df = pd.merge(df, w_df[["user_id", "task_container_id", "elapsed_time"]])

In [ ]:
df["timediff"] = df["timestamp"] - df.groupby(["user_id"])["timestamp"].shift()
df["timediff"] = df["timediff"].replace(0, np.nan).fillna(method="ffill") / df.groupby(["user_id", "task_container_id"])["user_id"].transform("count")

In [ ]:
df["timediff"] = [x if x < 100000 else np.nan for x in df["timediff"].values]

In [ ]:
df["elapsed_time_contentmean"] = df.groupby("content_id")["elapsed_time"].transform("mean")

In [ ]:
df["user_id"]

In [ ]:
df["elapsedtime_timediff"] = df["timediff"] - df["elapsed_time"]

In [ ]:
df["user_elapsed-timediff"] = df.groupby("user_id")["elapsedtime_timediff"].transform("mean")

In [ ]:
df[["task_container_id", "elapsed_time", "timediff", "elapsed_time_contentmean", "elapsedtime_timediff", "user_elapsed-timediff"]].head(10)

In [ ]:
df["timediff_mean"] = df.groupby(["content_id", "prior_question_had_explanation"])["timediff"].transform("mean")
df["timediff_median"] = df.groupby(["content_id", "prior_question_had_explanation"])["timediff"].transform("mean")

In [ ]:
df[["task_container_id", "elapsed_time", "timediff", "timediff_mean", "timediff_median", "elapsed_time_contentmean", "elapsedtime_timediff", "user_elapsed-timediff"]].head(10)

In [ ]:
df[["task_container_id", "elapsed_time", "timediff", "timediff_mean", "timediff_median", "elapsed_time_contentmean", "elapsedtime_timediff", "user_elapsed-timediff"]].tail(10)

In [ ]:
df["elapsedtime_timediff"].hist(bins=100, range=(0, 200000))

In [ ]:
df.head(100)